In [1]:
# data_fetcher.py
import pandas as pd
from fredapi import Fred
import wbdata
import datetime

# Replace 'YOUR_FRED_API_KEY' with the key you obtained
try:
    fred = Fred(api_key='3de73df1a97949f56207b1d2df009e12')
except Exception as e:
    print(f"Error initializing FRED API: {e}")
    print("Please ensure you have replaced 'YOUR_FRED_API_KEY' with a valid key.")
    fred = None

# --- FRED Data ---
def get_fred_series_latest(series_id):
    """Fetches the latest value for a FRED series."""
    if not fred:
        return "FRED API not initialized."
    try:
        data = fred.get_series(series_id)
        if data.empty:
            return f"No data found for FRED series {series_id}."
        latest_value = data.iloc[-1] # Get the last row/value
        latest_date = data.index[-1].strftime('%Y-%m-%d') # Get the last date
        return f"{latest_value:.2f} (as of {latest_date})"
    except Exception as e:
        return f"Error fetching FRED series {series_id}: {e}"

# --- World Bank Data (Example - requires refinement for 'latest') ---
def get_world_bank_indicator_latest(indicator_code, country_iso2='US'):
    """Fetches the latest non-null value for a World Bank indicator."""
    try:
        # Define a reasonable time range (e.g., last 20 years)
        end_year = datetime.datetime.now().year
        start_year = end_year - 20
        data = wbdata.get_dataframe(
            {indicator_code: "Value"},
            country=country_iso2,
            data_date=(datetime.datetime(start_year, 1, 1), datetime.datetime(end_year, 1, 1)),
            convert_date=True
        )
        if data.empty:
            return f"No data found for WB indicator {indicator_code} for {country_iso2}."

        # Data often comes with NaNs, find the latest valid entry
        data = data.dropna().sort_index() # Remove NAs and sort by date (year)
        if data.empty:
            return f"No recent valid data found for WB indicator {indicator_code} for {country_iso2}."

        latest_value = data['Value'].iloc[-1]
        latest_date = data.index[-1].strftime('%Y') # World Bank annual data often just has year
        return f"{latest_value:.2f} (as of {latest_date})"

    except Exception as e:
        return f"Error fetching World Bank indicator {indicator_code}: {e}"


# --- Simple Definitions (Knowledge Base) ---
definitions = {
    "gdp": "Gross Domestic Product (GDP) is the total monetary or market value of all the finished goods and services produced within a country's borders in a specific time period.",
    "real gdp": "Real GDP is an inflation-adjusted measure that reflects the value of all goods and services produced by an economy in a given year, expressed in base-year prices.",
    "inflation": "Inflation is the rate at which the general level of prices for goods and services is rising, and subsequently, purchasing power is falling.",
    "cpi": "The Consumer Price Index (CPI) measures the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services.",
    "unemployment rate": "The unemployment rate is the percentage of the labor force that is jobless and actively looking for employment."
}

indicator_map = {
    # Map keywords to specific API codes and source
    "real gdp": {"source": "fred", "id": "GDPC1"},
    "unemployment rate": {"source": "fred", "id": "UNRATE"},
    "cpi": {"source": "fred", "id": "CPIAUCSL_PC1"}, # Monthly % Change
    # Add World Bank examples if needed (adjust function calls)
    # "gdp world bank": {"source": "worldbank", "id": "NY.GDP.MKTP.CD", "country": "US"}
}

def get_definition(indicator_keyword):
    """Retrieves the definition for a keyword."""
    return definitions.get(indicator_keyword.lower(), "Sorry, I don't have a definition for that indicator.")

# Example Usage (for testing)
if __name__ == "__main__":
    print("Testing FRED Real GDP:", get_fred_series_latest('GDPC1'))
    print("Testing FRED Unemployment:", get_fred_series_latest('UNRATE'))
    print("Testing GDP Definition:", get_definition('gdp'))
    # print("Testing WB GDP:", get_world_bank_indicator_latest('NY.GDP.MKTP.CD', 'US')) # Example for WB

Testing FRED Real GDP: 23526.08 (as of 2025-01-01)
Testing FRED Unemployment: 4.20 (as of 2025-04-01)
Testing GDP Definition: Gross Domestic Product (GDP) is the total monetary or market value of all the finished goods and services produced within a country's borders in a specific time period.
